In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

c:\Users\detle\anaconda3\lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
c:\Users\detle\anaconda3\lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [3]:
df_fesx = pd.read_csv("./data/fesx_M15_2020-06-20_F.US.DSXM20.scid_BarData.txt", sep=", ", engine = "python")
df_fesx["Datetime"] = pd.to_datetime(df_fesx.Date + "-" + df_fesx.Time,format="%Y/%m/%d-%H:%M:%S")
df_fesx = df_fesx[['Datetime','Open', 'High', 'Low', 'Last', 'Volume']].rename(columns={"Last":"Close"})#.set_index("Datetime")
df_fesx = df_fesx.rename(columns={"Last":"Close"})
df_fesx.head()

,Datetime,Open,High,Low,Close,Volume
0,2020-05-21 02:15:00,2917,2921,2916,2918,837
1,2020-05-21 02:30:00,2917,2921,2916,2920,354
2,2020-05-21 02:45:00,2921,2922,2920,2920,213
3,2020-05-21 03:00:00,2920,2920,2915,2919,293
4,2020-05-21 03:15:00,2919,2919,2916,2917,511


In [5]:
rolling_n = 3
min_diff = 0


diff_high = df_fesx.High.diff().reset_index()
diff_low = df_fesx.Low.diff()
diff_low.index = diff_low.index -(rolling_n)
diff_low = diff_low.reset_index()
diff_df = pd.merge(diff_high, diff_low, on="index", how="left").rename(columns={"High":"diff_high", "Low":"diff_low"})


# For every n elements: high(idx - n)> ... >high(idx - 1)>high(idx - 0) => diff(idx) < 0
# n consecutive elements with decreasing high: rolling(n).max() < 0
diff_df['high_max'] = diff_df.diff_high.rolling(rolling_n).max()
diff_df['low_min'] = diff_df.diff_low.rolling(rolling_n).min()

diff_df['Open'] = False

diff_df.loc[(diff_df.high_max < min_diff) & (diff_df.low_min > min_diff), 'Open'] = True


fig = go.Figure(data= [
    go.Scatter(x=df_fesx.index, y=df_fesx.High,name = 'High',mode="lines+markers", marker_color = "green"),
    go.Scatter(x=df_fesx.index, y=df_fesx.Low,name = 'Low',mode="lines+markers", marker_color = "red"),
    go.Scatter(x=df_fesx[diff_df.Open].index, y=df_fesx[diff_df.Open].High,name = 'Open',mode="markers", marker_color = "orange", marker_size = 2 ),
    
    go.Candlestick(x=df_fesx.index,
                open=df_fesx['Open'],
                high=df_fesx['High'],
                low=df_fesx['Low'],
                close=df_fesx['Close'],
                name = 'Candlestick'
                )
    ]
    )

fig.update_layout(
    xaxis_rangeslider_visible= False
)



for idx, row in df_fesx[diff_df.Open].iterrows():
   exact_point = False
   high = row['High']
   open=row['Open']
   #print(idx, high)    
   # Check if the next point after a open is the same value => Dont consider next point as a touch point
   if  high == df_fesx.loc[idx+1,'High']:
      continue
   # Check if the next point after a open is lower than the open
   elif high > df_fesx.loc[idx+1,'High']:
      df_touch_line =  df_fesx[(df_fesx.index > idx) & (df_fesx.High >= high)]
      if df_touch_line.shape[0] == 0:
         continue
      if df_touch_line.High.iloc[0] == high:
         exact_point = True
   # Check if the next point after a open is higher than the open
   elif high < df_fesx.loc[idx+1,'High']:     
      df_touch_line =  df_fesx[(df_fesx.index > idx) & (df_fesx.High <= high)]
      if df_touch_line.shape[0] == 0:
         continue
      if df_touch_line.High.iloc[0] == high:
         exact_point = True
   
   if not exact_point:
      # Linear interpolation
      y = df_touch_line.High.iloc[0]
      x = df_touch_line.iloc[0].name
      xo = x -1
      yo = df_fesx.loc[xo,'High']
      m = (y - yo)/(x - xo)
      dy = high - yo
      dx = dy/m
      next_idx = xo + dx
   else:
      next_idx = df_touch_line.iloc[0].name


   fig.add_trace(
      go.Scatter(x=[idx, next_idx], y=[high, high],name = 'level',mode="lines", marker_color = "red", showlegend=False)
   )
          

fig.show()

# Check 1060

In [9]:
df_fesx[diff_df.Open]

,Datetime,Open,High,Low,Close,Volume
60,2020-05-21 17:15:00,2901,2901,2887,2895,51031
423,2020-05-28 09:15:00,3057,3061,3047,3061,28241
434,2020-05-28 12:00:00,3049,3054,3044,3052,14008
586,2020-06-01 10:30:00,3064,3067,3059,3061,13329
693,2020-06-02 17:30:00,3145,3151,3141,3149,123414
801,2020-06-04 05:00:00,3241,3242,3239,3241,1656
828,2020-06-04 11:45:00,3237,3238,3231,3236,19878
839,2020-06-04 14:30:00,3260,3263,3224,3228,67235
911,2020-06-05 12:45:00,3309,3311,3304,3309,12304
926,2020-06-05 16:30:00,3359,3364,3352,3358,35847
